# Качаем комментарии с youtube 

In [276]:
import requests
import pickle

from tqdm import tqdm_notebook

# Для распараллеливания кода
from joblib import Parallel, delayed

* https://medium.com/greyatom/youtube-data-in-python-6147160c5833
* https://console.developers.google.com/apis/api/youtube.googleapis.com/quotas?project=hopeful-vim-228218

In [177]:
api_key = ###

В коде страницы искать id по ссылке как: "externalId":

In [247]:
def get_all_video_in_channel(channel_id, api_key):
    """
    Достаёт словарь из id и имен всех видео с канала по его id
        channel_id : string
            id канала
        api_key : string
            ключ доступа к API
    """

    base_search_url = 'https://www.googleapis.com/youtube/v3/search?'
    
    first_url = base_search_url+'key={}&channelId={}&part=snippet,id&order=date&maxResults=25'.format(api_key, channel_id)

    video_links = { }
    url = first_url
    while True:
        req = requests.get(url)
        resp = req.json()

        for i in resp['items']:
            if i['id']['kind'] == "youtube#video":
                video_links.update({i['id']['videoId'] : i['snippet']['title']})

        try:
            next_page_token = resp['nextPageToken']
            url = first_url + '&pageToken={}'.format(next_page_token)
        except:
            break
    return video_links


def get_all_video_in_playlist(playlist_id, api_key):
    """
    Достаёт словарь из id и имен всех видео с плейлиста по его id
        playlist_id : string
            id канала
        api_key : string
            ключ доступа к API
    """

    base_search_url = 'https://www.googleapis.com/youtube/v3/playlistItems?'
    
    first_url = base_search_url+'key={}&playlistId={}&part=id,snippet&order=date&maxResults=25'.format(api_key, playlist_id)

    video_links = { }
    url = first_url
    while True:
        req = requests.get(url)
        resp = req.json()

        for i in resp['items']:
            if i['snippet']['resourceId']['kind'] == "youtube#video":
                video_links.update({i['snippet']['resourceId']['videoId'] : i['snippet']['title']})

        try:
            next_page_token = resp['nextPageToken']
            url = first_url + '&pageToken={}'.format(next_page_token)
        except:
            break
    return video_links


def get_comment_threads(video_id, api_key, style, performer, video_dict, limit=1000):
    """
    Достёт из-под видео 500 комментов
        video_id : string 
            id видяшки
        api_key : string
            ключ доступа к API 
        style : string
            стиль музыки 
        performer : string
            исполнитель
        video_divt : dict
            словарь с соответствием id и названий видео
    """

    base_search_url = 'https://www.googleapis.com/youtube/v3/commentThreads?'
    first_url = base_search_url+'key={}&videoId={}&part=snippet&maxResults=100&textFormat=plainText'.format(api_key, video_id)

    com_inf = [ ]
    url = first_url

    while len(com_inf) < limit:
        req = requests.get(url)
        resp = req.json()

        for item in resp['items']:
            comment = item["snippet"]["topLevelComment"]
            if comment['kind'] == 'youtube#comment':
                cur_infa = {
                    'music_style' : style,
                    'performer' : performer,
                    'video_name' : video_dict[video_id],
                    'comment_id' : comment["id"],
                    'author' : comment["snippet"]["authorDisplayName"],
                    'text' : comment["snippet"]["textDisplay"],
                    'date' : comment["snippet"]['publishedAt'],
                    'likes' : comment["snippet"]['likeCount']
                }

                com_inf.append(cur_infa)       
        try:
            next_page_token = resp['nextPageToken']
            url = first_url + '&pageToken={}'.format(next_page_token)
        except:
            break    
    return com_inf

In [270]:
style = 'popsa'

target = {
    "loboda" : {'канал' : "UCiU8w7NabK3xFgjxmHovjCg", 'листы' : ["PL2944ACC2EC785BD5"]},
    'билан' : {'канал' : "UCrnxeDn1Biodm9jO8yAyj3g", 'листы' : ["PL57Ql2OzGW1BDEuMJ3qi6fbzYON26fUTU"]},
    'темникова' : {'канал' : "UC4EiMlCetZkEOfFK8pfRAvw", 'листы' : ["PLQ7t2Z77CsoW6oeoaMxa665eO7hWA4s-t"]},
    'serebro' : {'канал' : "UCE3faOl-pvohsu7hof1Fnsw", 'листы' : ["PLcH8mHDiTxZ0DadRGyfEpSZFA6RZ203Ln"]},
    'гагарина' : {'канал' : "UCs6h0i7chZZX3fWD4IOh1kQ", 'листы' : ["PLjtCcgIBP_Ttn3N5JvdhQRBQGqrNTaQwK"]},
    'ёлка' : {'канал' : "UC3-HSETEZNYumQ4GXUiQAuw", 'листы' : [ ]},
    'виа_гра' : {'канал' : "UCgpnEoslcNaqYuf_78avHXw", 'листы' : ["PLmR3vfNp_YiO_fNanIM1ym-l8qqRbY0-i"]},
    'бузова' : {'канал' : "UCzTom8k7UduI9jcs6umWD9Q", 'листы' : ["PLqM15F9IQUAFbQW1XjBY3c7trFyOZ9Uol"]},
    'караулова' : {'канал' : "UCDGjJgCgyrbMfyjBzuo1xiA", 'листы' : ["PLP2A5cnvW__Ur_S0EOLCoLRyOZzUdCaFm"]},
    'кока' : {'канал' : "UCbj1rZy7-qktxZTawl1S_nA", 'листы' : ["PLdnFu7fbWUIMRoI5lFpait4mxL1JzU7Go"]},
    'нюша' : {'канал' : "UCm9VWKAFz0aXpuEHPHMae7w", 'листы' : ["PLJIRUPzMCrKKvenaGgQmjCDo49o7YByyT"]},
    'макsим' : {'канал' : "UCUy_4pyOCyQK7VM6pR5WxSw", 'листы' : ["PL6E039C93D2424F75"]},
    'градусы' : {'канал' : "UCBKvd5NasOgVXB1JmCL1urg", 'листы' : ["PLHb8DB_PqBc9fypbfTj0Hh41DP3CtwDnZ"]},
    'лазарев' : {'канал' : "UCRSZzzNedo9M2629ZTKBANA", 'листы' : ["PLVwU9Hicy4XVXQ6uHamkBfDygMbN53Tx7"]},
    'барских' : {'канал' : "UCSZP_I8E-_pRFVm90S0Zy6Q", 'листы' : ["PLyqQ1XMQ458rhd1QaXWhkX3YLuEO9CRTO"]},
    'крид' : {'канал' : "UCZeinbsBPa37qf4HPj_w-lw", 'листы' : ["PLomtNzMp4OgW3gmEb-IO35ojDS83gRgAe"]},
    'тимати' : {'канал' : "UCJjMGnyycI7f4Vl_UMuDB1Q", 'листы' : ["PLFBB9F68AF5C4F818"]},
    'руки_вверх' : {'канал' : "UCf5XmQY79pqR3i2oliGqMQg", 'листы' : ["PLLf5zOiu21LGS51xGPeHeDgO8H8QIPtbs"]},
    'astudio' : {'канал' : "UC0cst8k3zK9A8BRwZ4tE3AA", 'листы' : ["PLA9467A4A29649490"]},
    'iowa' : {'канал' : "UCj2K424HxCZTLWuw-PLBGNA", 'листы' : ["PLy3FBKe3LReDB5zQMMU9F9TB2_XjlQW3n"]},
    '2маши' : {'канал' : "UCt2p-g_IT0F_yuCfz79yKRw", 'листы' : ["PLrG9pIP1c-m9fyzfQfvFMVXHedc8LO-S5"]}
    }

In [267]:
style = 'estrada'

target = {
    'киркоров' : {'канал' : "UCJgUXBCxgiWUivJV2RmLD6g", 'листы' : ["PL1T7-Rtn2AzkT55xxmXmC01gKGXb0Br-b"]},
    'лепс' : {'канал' : "UCWopWNCej06Z3S4IG39yHLA", 'листы' : ["PLfBFsBXvPtQ5UCjScTwQxR2i00rKXN9ZM", "PLQuC5fFp9MxdUV0tNpkYcQMskJFhoFlN0"]},
    'пугачёва' : {'канал' : "UCtLuREHRfDQbKAzlomUvgzQ", 'листы' : [ ]},
    'михайлов': {'канал' : "UCc_rpSa6PXPow6aVRhlnQ8g", 'листы' : ["PLA360E570CE8C6566"]},
    'меладзе': {'канал' : "UCQKQrhLrtENPdxCNjeYUReg", 'листы' : [ ]},
    'ваенга': {'канал' : "UCyAcXWJxwjraTlzckEbgMfQ", 'листы' : ["PLS6Hv_KxWNTxZtQ3PyyfZNZaxW7D02rL8"]},
    'валерия': {'канал' : "UC8ctItMhn_FNS1c301_Q-zA", 'листы' : ["PLapk1eIugjg4wTQYSsAPNz9ZQnaH8ijSP"]},
    'агутин': {'канал' : "UCVafaAr4oDm-DurxwclK3Zg", 'листы' : [ ]},
    'басков': {'канал' : "UCJqsNPrsH71EnsP8kbtrHoA", 'листы' : ["PLU3z5rOtm15b41T47fJiX1GRqDA1LVbcP"]},
    'ротару': {'канал' : "UC0Ux11GJjRsXA-DbU6zUMkw", 'листы' : [ ]},
    'лолита' : {'канал' : "UCaKeasZJuQ_ZYw0o9hsfGbg", 'листы' : ["PL7Mdigi3VDUd2hkYvisfOEojlzkO3XZp4"]},
    'газманов': {'канал' : "UCxPUxaXn8sVLABf7HxxMabQ", 'листы' : ["PL7yxEFBiOVQYSFwc9Rn_G1tHM2epv5IxH"]},
    'николаев': {'канал' : "UC9nYweZwDnAr-kIkADlJA6A", 'листы' : ["PL23vzKgeCvbwQVzrR23jSSV5G9D_KHOOX"]},
    'леонтьев': {'канал' : "UCi0x0GBJ7095DM4qjTIwp9w", 'листы' : [ ]},
    'буйнов': {'канал' : "UCRlcuihl8_Qp657AYTC_KUA", 'листы' : [ ]}      
    }


In [268]:
comments_dict = { }

In [269]:

for performer in target.keys():
#for performer in ['басков', 'лолита']:
    print('Качаю {}'.format(performer))
    
    channel_id = target[performer]['канал']
    playlist_ids =  target[performer]['листы']

    # видосы с основного канала 
    video_chanel = get_all_video_in_channel(channel_id, api_key)

    # видосы с плейлистов
    video_playlist = { }
    for lst in playlist_ids:
        video_playlist.update(get_all_video_in_playlist(lst, api_key))

    print('Видео с канала:',len(video_chanel))
    print('Видео с плейлиста:',len(video_playlist))
    video_playlist.update(video_chanel)
    print('Видео всего:',len(video_playlist))
    
    # пошли комменты
    def getCom(x):
        try:
            return get_comment_threads(x, api_key, style, performer, video_playlist)
        except:
            print('Error in https://www.youtube.com/watch?time_continue=1&v={}'.format(x))
            return [ ]

    n_jobs = -1 # параллелим на все ядра 
    result = Parallel(n_jobs=n_jobs)(delayed(getCom)(
        text) for text in tqdm_notebook(list(video_playlist.keys())))
    
    comments = [ ]
    for item in result:
        comments.extend(item)

    print('Добыто комментариев', len(comments))
    
    # воткнуть сюда сохранялку
    comments_dict[performer] = comments
    print("==================================================")

Качаю киркоров
Видео с канала: 334
Видео с плейлиста: 189
Видео всего: 453


HBox(children=(IntProgress(value=0, max=453), HTML(value='')))

Error in https://www.youtube.com/watch?time_continue=1&v=W-27R8SBu1A
Добыто комментариев 18207
Качаю лепс
Видео с канала: 386
Видео с плейлиста: 222
Видео всего: 480


HBox(children=(IntProgress(value=0, max=480), HTML(value='')))

Error in https://www.youtube.com/watch?time_continue=1&v=9Ht3f1WPG4g
Error in https://www.youtube.com/watch?time_continue=1&v=wG28KFqsJQA
Error in https://www.youtube.com/watch?time_continue=1&v=4uDNofw0aG8
Error in https://www.youtube.com/watch?time_continue=1&v=FZqbkPvrUIo
Добыто комментариев 12561
Качаю пугачёва
Видео с канала: 387
Видео с плейлиста: 0
Видео всего: 387


...

Error in https://www.youtube.com/watch?time_continue=1&v=k-hWXEW4Wd0
Error in https://www.youtube.com/watch?time_continue=1&v=W0JtOMzN96o
Error in https://www.youtube.com/watch?time_continue=1&v=s7lRuXX2H8I
Error in https://www.youtube.com/watch?time_continue=1&v=NlQDhYGWUmc
Добыто комментариев 3645
Качаю леонтьев
Видео с канала: 76
Видео с плейлиста: 0
Видео всего: 76


HBox(children=(IntProgress(value=0, max=76), HTML(value='')))

Добыто комментариев 3
Качаю буйнов
Видео с канала: 43
Видео с плейлиста: 0
Видео всего: 43


HBox(children=(IntProgress(value=0, max=43), HTML(value='')))

Добыто комментариев 0


In [277]:
# Путь к данным
path = '/Users/fulyankin/Yandex.Disk.localized/Проекты/top_russian_music/comments/youtube/by_style'

with open(path + 'youtube_popsa', 'wb') as f:
    pickle.dump(comments_dict, f)

In [271]:
full_comments = [ ]
for item in comments_dict:
    full_comments.extend(comments_dict[item])
    
len(full_comments)

100713

In [272]:
full_comments[40000]

{'author': 'Толя друг',
 'comment_id': 'UgzNDKFueNMZ1O8Ie4N4AaABAg',
 'date': '2018-06-23T09:40:17.000Z',
 'likes': 1,
 'music_style': 'estrada',
 'performer': 'михайлов',
 'text': 'Круто',
 'video_name': 'Стас Михайлов и Елена Север - Не зови, не слышу (LIVE RU TV 2017)'}

In [273]:
comments_dict['леонтьев']

[{'author': 'qalina rtty',
  'comment_id': 'UgxbyUad0DaDsKvho9h4AaABAg',
  'date': '2019-01-02T12:07:59.000Z',
  'likes': 0,
  'music_style': 'estrada',
  'performer': 'леонтьев',
  'text': 'классно!!! но жаль, что без видео.',
  'video_name': 'Старый город в ритме дождя'},
 {'author': 'ipr vrn',
  'comment_id': 'Ugz5suvHJnz3_Gc_WZV4AaABAg',
  'date': '2018-12-20T02:40:16.000Z',
  'likes': 0,
  'music_style': 'estrada',
  'performer': 'леонтьев',
  'text': 'Именно та запись, что слушал 35 лет тому... 👍',
  'video_name': 'Там, в сентябре'},
 {'author': 'John Cartwright',
  'comment_id': 'UgyQm8pcV5ockJjNWjp4AaABAg',
  'date': '2018-12-04T23:15:51.000Z',
  'likes': 0,
  'music_style': 'estrada',
  'performer': 'леонтьев',
  'text': 'Stalker OLR 2.5 brought me here.',
  'video_name': 'Flight on a hang glider'}]